# Image Classification from scratch with TPUs on Cloud ML Engine using ResNet

This notebook demonstrates how to do image classification from scratch on a flowers dataset using TPUs and the resnet trainer.

In [ ]:
import os

GC_PROJECT =  # REPLACE WITH YOUR PROJECT ID
BUCKET = 'product-classification' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
PROJECT = 'project_001_freisteller'
MODEL_VERSION = 'ResNet_v03'

# do not change these
os.environ['GC_PROJECT'] = GC_PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['PROJECT'] = PROJECT
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TFVERSION'] = '1.14'

In [ ]:
%%bash
gcloud config set project $GC_PROJECT
gcloud config set compute/region $REGION

## Enable TPU service account

Allow Cloud ML Engine to access the TPU and bill to your project

In [ ]:
%%bash
SVC_ACCOUNT=$(curl -H "Authorization: Bearer $(gcloud auth print-access-token)"  \
    https://ml.googleapis.com/v1/projects/${GC_PROJECT}:getConfig \
              | grep tpuServiceAccount | tr '"' ' ' | awk '{print $3}' )
              
echo "Enabling TPU service account $SVC_ACCOUNT to act as Cloud ML Service Agent"
gcloud projects add-iam-policy-binding $GC_PROJECT --member serviceAccount:$SVC_ACCOUNT --role roles/ml.serviceAgent



## Try preprocessing locally

In [ ]:
!rm -r ${PWD}/local_test/*

In [ ]:
%%bash
gsutil cat gs://${BUCKET}/${PROJECT}/train_set.csv | head -5 > ${PWD}/local_test/input.csv
gsutil cat gs://${BUCKET}/${PROJECT}/labels.txt | sort > ${PWD}/local_test/labels.txt

In [ ]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/mymodel

python -m trainer.preprocess \
       --train_csv ${PWD}/local_test/input.csv \
       --validation_csv ${PWD}/local_test/input.csv \
       --labels_file ${PWD}/local_test/labels.txt \
       --project_id $PROJECT \
       --output_dir ${PWD}/local_test/out \
       --runner=DirectRunner

In [ ]:
!ls -l ${PWD}/local_test/out

Now run it over full training and evaluation datasets.  This will happen in Cloud Dataflow.

In [ ]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/mymodel
    
gsutil -m rm -rf gs://${BUCKET}/resnet/data
python -m trainer.preprocess \
       --train_csv gs://${BUCKET}/${PROJECT}/train_set.csv \
       --validation_csv gs://${BUCKET}/${PROJECT}/eval_set.csv \
       --labels_file gs://${BUCKET}/${PROJECT}/labels.txt \
       --project_id $GC_PROJECT \
       --output_dir gs://${BUCKET}/${PROJECT}/resnet/data

https://console.cloud.google.com/dataflow

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/${PROJECT}/resnet/data

## Train on the Cloud

In [ ]:
%%bash
echo -n "--num_train_images=$(gsutil cat gs://${BUCKET}/${PROJECT}/train_set.csv | wc -l)  "
echo -n "--num_eval_images=$(gsutil cat gs://${BUCKET}/${PROJECT}/eval_set.csv | wc -l)  "
echo "--num_label_classes=$(gsutil cat gs://${BUCKET}/${PROJECT}/labels.txt | wc -l)"

In [ ]:
%%bash
TOPDIR=gs://${BUCKET}/${PROJECT}/resnet
OUTDIR=${TOPDIR}/trained
JOBNAME=imgclass_$(date -u +%y%m%d_%H%M%S)


echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR  # Comment out this line to continue training from the last time

gcloud ai-platform jobs submit training $JOBNAME \
 --region=$REGION \
 --module-name=trainer.resnet_main \
 --package-path=$(pwd)/mymodel/trainer \
 --job-dir=$OUTDIR \
 --staging-bucket=gs://$BUCKET \
 --scale-tier=BASIC_TPU \
 --runtime-version=$TFVERSION --python-version=3.5 \
 -- \
 --data_dir=${TOPDIR}/data \
 --model_dir=${OUTDIR}/ \
 --resnet_depth=18 \
 --train_batch_size=128 --eval_batch_size=64 --skip_host_call=True \
 --steps_per_eval=150 --train_steps=500 \
 --num_train_images=865 --num_eval_images=94 --num_label_classes=6 \
 --export_dir=${OUTDIR}/export

Run this on CLOUD Command:
    tensorboard --logdir=gs://product-classification/project_001_freisteller/resnet/trained/ --port=8000

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/${PROJECT}/resnet/trained

## Deploying and predicting with model

Deploy the model:

In [ ]:
%%bash
MODEL_NAME=${PROJECT}
TOPDIR=gs://${BUCKET}/${PROJECT}/resnet
OUTDIR=${TOPDIR}/trained
JOBNAME=${PROJECT}_$(date -u +%y%m%d_%H%M%S)


MODEL_VERSION=${MODEL_VERSION}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/${PROJECT}/resnet/trained/export/ | tail -1)
echo "Deleting/deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"

# comment/uncomment the appropriate line to run. The first time around, you will need only the two create calls
# But during development, you might need to replace a version by deleting the version and creating it again

#gcloud ml-engine versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION 

We can use saved_model_cli to find out what inputs the model expects:

As you can see, the model expects image_bytes.  This is typically base64 encoded

In [ ]:
%%bash
head ${PWD}/local_test/labels.txt 

In [ ]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import base64, sys, json
import tensorflow as tf
import ast
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


picture = '17881084.jpg'

with open('/home/jupyter/' + PROJECT + '/local_test/labels.txt', 'r') as f:
    lines = f.read().splitlines()

with tf.gfile.GFile('gs://' + BUCKET + '/' + PROJECT + '/images/'+ picture, 'rb') as ifp:
    credentials = GoogleCredentials.get_application_default()
    api = discovery.build('ml', 'v1', credentials=credentials, discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')
    
    request_data = {'instances':
                    [
                        {"input": {"b64": base64.b64encode(ifp.read()).decode('utf-8')}}
                    ]}
    parent = 'projects/%s/models/%s/versions/%s' % (GC_PROJECT, PROJECT, MODEL_VERSION)
    response = api.projects().predict(body=request_data, name=parent).execute()


    img=mpimg.imread('/home/jupyter/' + BUCKET + '/' + PROJECT + '/images/' + picture)
    imgplot = plt.imshow(img)
    plt.show()
    
    
    print(lines[response['predictions'][0]['probabilities'].index(max(response['predictions'][0]['probabilities']))])
    
    print(lines)
    print(response)